In [1]:
!pip install albumentations
!pip install ultralytics
!pip install flask
!pip install scikit-learn

     ---------------------------------------- 0.0/146.9 kB ? eta -:--:--
     -------------------------------------- 146.9/146.9 kB 4.4 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/178.0 kB ? eta -:--:--
   ---------------------------------------- 178.0/178.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
    --------------------------------------- 0.7/38.8 MB 22.1 MB/s eta 0:00:02
   - -------------------------------------- 1.6/38.8 MB 20.3 MB/s eta 0:00:02
   --- ------------------------------------ 2.9/38.8 MB 26.7 MB/s eta 0:00:02
   ---- ----------------------------------- 4.6/38.8 MB 32.8 MB/s eta 0:00:02
   ------- -------------------------------- 7.0/38.8 MB 37.6 MB/s eta 0:00:01
   ---------- ----------------------------- 10.5/38.8 MB 50.4 MB/s eta 0:00:01
   --------------- ------------------------ 14.6/38.8 MB 93.0 MB/s eta 0:00:01
   -----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     ---------------------------------------- 0.0/41.9 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.9 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 41.9/41.9 kB 673.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/871.8 kB ? eta -:--:--
   -------------------- ------------------ 450.6/871.8 kB 14.2 MB/s eta 0:00:01
   --------------------------------------- 871.8/871.8 kB 13.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.3 MB ? eta -:--:--
   ---------------------------------------- 0.9/199.3 MB 29.4 MB/s eta 0:00:07
   ------------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.7 kB ? eta -:--:--
   ------------------------------------ --- 92.2/101.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 1.5 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

# Libraries for Data Preprocessing
import os
import cv2
import albumentations as A
import numpy as np

#Libraries for data splitting
import shutil
from sklearn.model_selection import train_test_split

# Libraries for ML Model
from ultralytics import YOLO

# Libraries for Python Web App
from flask import Flask, request, render_template, send_file

True
0
NVIDIA GeForce RTX 3070 Ti


In [2]:
# Function for data splitting
def custom_train_test_split(dataset_directory, test_ratio=0.2, random_seed=42, min_samples_per_class=2):
    
    class_directories = [d for d in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, d))]
    
    train_directory = os.path.join(dataset_directory, 'train')
    test_directory = os.path.join(dataset_directory, 'test')
    os.makedirs(train_directory, exist_ok=True)
    os.makedirs(test_directory, exist_ok=True)

    for class_dir in class_directories:
        class_path = os.path.join(dataset_directory, class_dir)
        image_files = [f for f in os.listdir(class_path) if f.endswith('.jpg')]

        if len(image_files) < min_samples_per_class:
            continue

        train_images, test_images = train_test_split(image_files, test_size=test_ratio, random_state=random_seed)

        for train_image in train_images:
            src_image_path = os.path.join(class_path, train_image)
            src_annotation_path = os.path.join(class_path, train_image.replace('.jpg', '.txt'))

            dest_image_path = os.path.join(train_directory, class_dir, train_image)
            dest_annotation_path = os.path.join(train_directory, class_dir, train_image.replace('.jpg', '.txt'))

            os.makedirs(os.path.dirname(dest_image_path), exist_ok=True)

            if os.path.exists(src_annotation_path):
                shutil.copy(src_image_path, dest_image_path)
                shutil.copy(src_annotation_path, dest_annotation_path)

        for test_image in test_images:
            src_image_path = os.path.join(class_path, test_image)
            src_annotation_path = os.path.join(class_path, test_image.replace('.jpg', '.txt'))

            dest_image_path = os.path.join(test_directory, class_dir, test_image)
            dest_annotation_path = os.path.join(test_directory, class_dir, test_image.replace('.jpg', '.txt'))

            os.makedirs(os.path.dirname(dest_image_path), exist_ok=True)

            if os.path.exists(src_annotation_path):
                shutil.copy(src_image_path, dest_image_path)
                shutil.copy(src_annotation_path, dest_annotation_path)

# Executing function for Data Splitting
dataset_directory = './dataset'
custom_train_test_split(dataset_directory, test_ratio=0.2, random_seed=42, min_samples_per_class=2)

In [3]:
# Define the augmentation pipeline with specified techniques
transform = A.Compose([
    A.Resize(640, 640),                                         # Resize images to 640x640
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),     # Normalize images (mean and std values can be adjusted)
    A.RandomRotate90(p=0.5),                                    # Randomly rotate 90 degrees
    A.Flip(p=0.5),                                              # Randomly flip the images
    A.Transpose(p=0.5),                                         # Randomly transpose the images
    A.RandomBrightnessContrast(p=0.5),                          # Adjust brightness and contrast randomly
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),                # Add Gaussian noise
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def preprocess_and_augment(image_path, annotation_path, output_dir, transform):
    """
    Applies resizing, normalization, and augmentation to images and adjusts bounding boxes accordingly.
    Saves the processed images and adjusted annotations.
    """
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Read the bounding boxes from the annotation file
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    bboxes = []
    class_labels = []
    # Parsing bounding boxes in YOLO format [class, center_x, center_y, width, height]
    for line in lines:
        class_number, centre_x, centre_y, width, height = map(float, line.strip().split())
        bboxes.append([centre_x, centre_y, width, height])
        class_labels.append(int(class_number))

    # Apply augmentation and get the transformed image and bounding boxes
    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    augmented_image = augmented['image']
    augmented_bboxes = augmented['bboxes']
    augmented_labels = augmented['class_labels']

    # Save the augmented image
    augmented_image_path = os.path.join(output_dir, os.path.basename(image_path).replace('.jpg', '_aug.jpg'))
    cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    # Save the adjusted bounding boxes
    augmented_annotation_path = augmented_image_path.replace('.jpg', '.txt')
    with open(augmented_annotation_path, 'w') as file:
        for bbox, label in zip(augmented_bboxes, augmented_labels):
            file.write(f"{label} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

# Define paths to dataset and output directory
output_directory = './augemented'
os.makedirs(output_directory, exist_ok=True)

# Iterate over each class subdirectory and augment images
for class_name in ['glioma', 'meningioma', 'pituitary', 'notumor']:  # Ensure these match your folder names
    class_directory = os.path.join(dataset_directory, class_name)
    image_files = [f for f in os.listdir(class_directory) if f.endswith('.jpg')]

    for image_file in image_files:
        image_path = os.path.join(class_directory, image_file)
        annotation_path = image_path.replace('.jpg', '.txt')

        # Ensure the annotation file exists before processing
        if os.path.exists(annotation_path):
            preprocess_and_augment(image_path, annotation_path, output_directory, transform)


In [4]:
# Load the YOLOv8 model configuration (adjust model size: 'yolov8n.yaml', 'yolov8s.yaml', etc.)
model = YOLO('yolov8s.yaml')  # Choose the appropriate model variant

model.to('cuda')

# Train the model using the data configuration file
model.train(data='./brain_tumor_dataset.yaml', 
            epochs=50, 
            imgsz=640, 
            batch=16, 
            project='./models')  # Adjust epochs and batch size as needed

engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=./brain_tumor_dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=./models, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, 

train: Scanning C:\Users\EvasiveXkiller\Desktop\Brain-Tumour-Detection-and-Classification-using-YOLOv8\dataset\train\glioma.cache... 1740 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1740/1740 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\EvasiveXkiller\Desktop\Brain-Tumour-Detection-and-Classification-using-YOLOv8\dataset\test\glioma.cache... 435 images, 0 backgrounds, 0 corrupt: 100%|██████████| 435/435 [00:00<?, ?it/s]


Plotting labels to models\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to models\train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        435        439      0.911      0.861      0.922      0.561



50 epochs completed in 0.268 hours.
Optimizer stripped from models\train7\weights\last.pt, 22.5MB
Optimizer stripped from models\train7\weights\best.pt, 22.5MB

Validating models\train7\weights\best.pt...
Ultralytics YOLOv8.2.90  Python-3.9.19 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
YOLOv8s summary (fused): 168 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  4.79it/s]


                   all        435        439      0.891      0.882      0.927      0.563
             pituitary        124        125      0.916      0.928      0.969      0.513
            meningioma        110        110      0.937      0.964      0.986      0.623
                glioma         91         94       0.83      0.691      0.773       0.38
               notumor        110        110      0.879      0.945      0.978      0.736
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to models\train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000203E2A4DAC0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

In [7]:
# Save the model weights
torch.save(model.state_dict(), './models/exported.pth')

In [ ]:
app = Flask(__name__)
model = YOLO('path_to/best.pt')  # Load your trained model weights

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        file = request.files['file']
        image_path = os.path.join('uploads', file.filename)
        file.save(image_path)

        # Predict and draw bounding boxes
        results = model.predict(image_path, save=True, imgsz=640, conf=0.3)
        output_image_path = results[0].path  # Path to saved prediction image

        return send_file(output_image_path, mimetype='image/jpeg')  # Display the processed image

    return render_template('upload.html')  # Render a simple HTML upload form

if __name__ == '__main__':
    app.run(debug=True)